In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from rp_utils import costFunc, actFunc, initFunc, regFunc, decayFunc, batchGenFunc, utils
import rosenpymodel.cvffnn as mynn1
import rosenpymodel.scffnn as mynn2
import rosenpymodel.cvrbfnn as mynn3
import rosenpymodel.fcrbfnn as mynn4
import rosenpymodel.deepptrbfnn as mynn5
import numpy as np
from sklearn.datasets import load_iris
from matplotlib import cm
from matplotlib.colors import ListedColormap

In [ ]:
x, y = make_spiral(n_samples=100, n_class=2, radius=5, laps=1.75)

z = np.array([x[:,0] + x[:,1]*1j], ndmin=1).T

y = y.astype(int)

plt.scatter(z.real, z.imag, c=list(np.array(y).ravel()), s=15, cmap=dsa())

y1 = initFunc.zeros(100, 1)
y2 = initFunc.ones(100, 1)
yz = np.concatenate((y1,y2))


# CVFFNN

In [ ]:
##CVFFNN

nn1 = mynn1.CVFFNN(cost_func=costFunc.mse,  learning_rate = 1e-3)
nn1.addLayer(ishape=z.shape[1], neurons=40, activation=actFunc.atan, bias_initializer=initFunc.zeros)
nn1.addLayer(neurons=20, activation=actFunc.atanh)
nn1.addLayer(neurons=y.shape[1], activation=actFunc.atan, bias_initializer=initFunc.zeros)

hist1 = nn1.fit(z, y, epochs=5000, verbose=100)

y_pred1 = nn1.predict(z)


arr = y_pred1.view(float)
m = arr % 1. <= .5
arr[m] = np.floor(arr[m])
arr[~m] = np.ceil(arr[~m])
arr1 = arr.view(complex)

print('Accuracy CVFFNN: {:.2f}%'.format(utils.accuracy(y, y_pred1)))


# SCFFNN

In [ ]:
##SCFFNN
nn2 = mynn2.SCFFNN(cost_func=costFunc.mse,  learning_rate = 1e-3)
nn2.addLayer(ishape=z.shape[1], neurons=100, activation=actFunc.tanh, bias_initializer=initFunc.zeros)
nn2.addLayer(neurons=50, activation=actFunc.tanh)
nn2.addLayer(neurons=y.shape[1], activation=actFunc.tanh, bias_initializer=initFunc.zeros)

hist2 = nn2.fit(z, y, epochs=5000, verbose=100)

y_pred2 = nn2.predict(z)

arr = y_pred2.view(float)
m = arr % 1. <= .5
arr[m] = np.floor(arr[m])
arr[~m] = np.ceil(arr[~m])
arr2 = arr.view(complex)

print('Accuracy SCFFNN: {:.2f}%'.format(utils.accuracy(y, y_pred2)))


# CVRBFNN

In [ ]:
nn3 = mynn3.CVRBFNN(learning_rate=1e-2)
nn3.addLayer(ishape=z.shape[1], neurons=60, oshape=yz.shape[1], 
            gamma_rate=0.1, sigma_rate=0.1)

hist3 = nn3.fit(z, yz, epochs=5000, verbose=100)

y_pred3 = nn3.predict(z)

arr = y_pred3.view(float)
m = arr % 1. <= .5
arr[m] = np.floor(arr[m])
arr[~m] = np.ceil(arr[~m])
arr3 = arr.view(complex)

In [ ]:
print('Accuracy CVRBFNN: {:.2f}%'.format(utils.accuracy(yz, y_pred3)))


# FCFFNN

In [ ]:
##FCFFNN

nn4 = mynn4.FCRBFNN(learning_rate=1e-3)
nn4.addLayer(ishape=z.shape[1], neurons=20, oshape=y.shape[1], 
               gamma_rate=0.001, sigma_rate=0.001)

hist4 = nn4.fit(z, y, epochs=5000, verbose=100)


y_pred4 = nn4.predict(z)

arr = y_pred4.view(float)
m = arr % 1. <= .5
arr[m] = np.floor(arr[m])
arr[~m] = np.ceil(arr[~m])
arr4 = arr.view(complex)


print('Accuracy CVRBFNN: {:.2f}%'.format(utils.accuracy(y, y_pred4)))

# DeepPTRBFNN

In [ ]:
##DeepPTRBFNN

nn5 = mynn5.DeepPTRBFNN(learning_rate=1e-3, momentum=0.1)
nn5.addLayer(ishape=z.shape[1], neurons=20, gamma_rate=1e-3, sigma_rate=1e-2)
nn5.addLayer(neurons=20, oshape=y.shape[1], gamma_rate=1e-2, sigma_rate=1e-2)

hist5 = nn5.fit(z, y, epochs=5000, verbose=100)

y_pred5 = nn5.predict(z)

arr = y_pred5.view(float)
m = arr % 1. <= .5
arr[m] = np.floor(arr[m])
arr[~m] = np.ceil(arr[~m])
arr5 = arr.view(complex)

print('Accuracy DeepPTRBFNN: {:.2f}%'.format(utils.accuracy(y, y_pred5)))

# Real

In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(50,
                                activation=tf.nn.relu,
                                input_shape=(x.shape[1],)))
model.add(tf.keras.layers.Dense(50, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(50, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
model.summary()
model.compile(optimizer='SGD',
              loss='binary_crossentropy',
              metrics=['accuracy'])
hist = model.fit(x,
                 y,
                 epochs=5001,
                 verbose=0)

In [ ]:
x=np.arange(0, 5001, 100)

plt.figure(dpi=100)
plt.rcParams['savefig.dpi'] = 150
plt.plot( hist1['epochs'][0::2], hist1['loss'][0::2], 'go-', label='CV-FFNN') # green 
plt.plot( hist2['epochs'][0::2], hist2['loss'][0::2], 'yo-', label='SC-FFNN')  # yellow 
plt.plot( hist3['epochs'][0::2], hist3['loss'][0::2], 'bo-', label='CV-RBFNN') # blue 
plt.plot( hist4['epochs'][0::2], hist4['loss'][0::2], 'mo-', label='FC-RBFNN') # magenta 
plt.plot( hist5['epochs'][0::2], hist5['loss'][0::2], 'co-', label='Deep PT-RBFNN') # cyan 
plt.plot( hist.epoch[0::200], hist.history['loss'][0::200], 'ro-', label='RVNN') # red 

plt.legend()
plt.grid(True)
plt.xlabel("epochs")
plt.ylabel("training loss")
plt.show()